# *Импорт библиотек*

In [ ]:
%%capture
!pip install autokeras

In [ ]:
from google.colab import files # Импортируем Модуль files для работы с файлами
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
%matplotlib inline
import seaborn as sns

plt.style.use('seaborn-white')
sns.set_context('notebook', rc={'figure.figsize': (10, 8)}, font_scale=1.5)

import numpy as np # Импортируем библиотеку numpy
import pandas as pd

import time
import random
import math
import re
import os # Импортируем библиотеку os для раоты с фаловой системой
import shutil
import json
import cv2

from copy import deepcopy
from IPython.display import clear_output

# from tensorflow.keras.applications import MobileNetV3Small
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import one_hot
# from tensorflow.keras.preprocessing import image # Импортируем модуль image для работы с изображениями

from sklearn.model_selection import train_test_split

<ipython-input-2-2f058e63d896>:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [ ]:
from PIL import Image, ImageFile, ImageOps # импортируем модель Image для работы с изображениями
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from google.colab import drive # Подключаем гугл-диск
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Контекстный менеджер для измерения времени операций
# Операция обертывается менеджером с помощью оператора with
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        # print('Время обработки: {:.2f} с'.format(time.time() - self.t))
        seconds_total = time.time() - self.t
        minutes = int(seconds_total // 60)
        seconds = (seconds_total % 60) + (seconds_total - math.floor(seconds_total))
        print('Время обработки: {} мин {:.2f} c'.format(minutes, seconds))

with timex():
    print()


Время обработки: 0 мин 0.01 c


In [ ]:
# Указываем путь к базе данных
directory = '/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/Scans'
directory_new = '/content/ClearedReshapedBase'

# **Подготовка данных**

In [ ]:
# Создаём выборку только из вертикальных изображений
files_dirs = []

# Выборка 500 изображений
# dont_load_index = [i-1 for i in [184, 185, 171, 157, 129, 12]] + [i+200-1 for i in [162, 136, 70, 48]] + [i+400-1 for i in [78,43]]
# 0,500

# Выборка 1000 изображений
# dont_load_index = [i-1 for i in [184, 185, 171, 157, 129, 12]] + [i+200-1 for i in [162, 136, 70, 48]] + [i+400-1 for i in [78,43]] + [i+500-1 for i in [39, 27, 25]] + [i+700-1 for i in [193,79,55]] + [i+900-1 for i in [83, 60]]
# 0,1000

# Выборка 250 изображений валидационная
dont_load_index = [1000-1 + i for i in [246,241,240,238,208,175,114,92,85,78,51,48,46,42,40,35,1]] + [1250-1 + i for i in [245,240,188,178,135,102]]
# 1000,1500

files_list = os.listdir(directory)
for i in range(1000,1500):
    if i not in dont_load_index:
        files_dirs.append(directory + '/' + files_list[i])

print(files_dirs[0])
print(len(files_dirs))

/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/Scans/2022_7_3_9_49_19_edbd1a08-4476-494f-bc96-64fefdb7b70a.jpg
477


In [ ]:
# Размер, к которому вы хотите привести изображения
target_size = (224, 224)

image_list = []
with timex():
    for filename in files_dirs:
        img = cv2.imread(filename)                                          # Открываем
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)                         # Переводим в чёрно-белый вид
        img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)    # Уменьшаем

        image_list.append(img)

# Поворот изображений и добавление их в общую выборку
rotated_images = []
with timex():
    for img in image_list:
        rotated_images.append(img)  # Оригинальное изображение

        # Поворот на 90, -90 и 180 градусов
        for angle_func in [cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]:
            rotated_images.append(cv2.rotate(img, angle_func))

del image_list

Время обработки: 1 мин 4.12 c
Время обработки: 0 мин 0.06 c


In [ ]:
rotated_images = [image / 255.0 for image in rotated_images]                    # нормализация (диапазон [0, 1])
rotated_images = [np.expand_dims(image, axis=2) for image in rotated_images]    # добавляем ещё одну размерность (224,224,1)
rotated_images = np.array(rotated_images)                                       # перевод в numpy массив

In [ ]:
# Разметка изображений
labels = [0, 1, 2, 3] * (len(rotated_images) // 4)

# Перевод меток в формат One-Hot Encoding
one_hot_labels = np.array(one_hot(labels, depth=4))

In [ ]:
# Создание обучающей и тестовой выборок
xTrain, xTest, yTrain, yTest = train_test_split(rotated_images, one_hot_labels, test_size=0.2, random_state=42)
del one_hot_labels, rotated_images

In [ ]:
# Для валидационной выборки
x = np.vstack((xTrain, xTest))
y = np.vstack((yTrain, yTest))

In [ ]:
import gc
gc.collect()

755

# **Обучение архитектуры autokeras**
> Точность на тестовой выборке: 0.9962


In [ ]:
import autokeras as ak

# Создаём ImageClassifier
clf = ak.ImageClassifier(max_trials=1)

In [ ]:
# Обучаем модель
clf.fit(xTrain, yTrain, epochs=5)

# Результаты модели
results = clf.evaluate(xTest, yTest)
print(results)

# Экспортируем модель
best_model = clf.export_model()
best_model.summary()

# Сохраняем модель
best_model.save('best_model_auto.h5')

Epoch 1/5
98/98 [==============================] - 669s 7s/step - loss: 0.1458 - accuracy: 0.9790
Epoch 2/5
98/98 [==============================] - 675s 7s/step - loss: 0.0120 - accuracy: 0.9971
Epoch 3/5
98/98 [==============================] - 661s 7s/step - loss: 0.0012 - accuracy: 0.9994
Epoch 4/5
98/98 [==============================] - 655s 7s/step - loss: 1.5716e-05 - accuracy: 1.0000
Epoch 5/5
25/25 [==============================] - 43s 2s/step - loss: 0.0066 - accuracy: 0.9962
[0.006616901606321335, 0.9961734414100647]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 cast_to_float32 (CastToFlo  (None, 224, 224, 1)       0         
 at32)                                                           
                                                     

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
best_model.save('/content/drive/MyDrive/bestmodel_autokeras.h5')

# **Проверка точности на валидационной выборке**
> Точность на проверочной выборке: 1.0000

In [ ]:
from autokeras import CUSTOM_OBJECTS
import gdown
import tensorflow as tf

Using TensorFlow backend


In [ ]:
# Загрузка модели нахождения поворота изображений на 90°, -90°, 180°
gdown.download(f"https://drive.google.com/uc?id=1ayQNQmPirwYVjTR7E5M4WFrvrclp7J8E", 'rotate_model.h5')
model = tf.keras.models.load_model("rotate_model.h5", custom_objects=CUSTOM_OBJECTS)

Downloading...
From: https://drive.google.com/uc?id=1ayQNQmPirwYVjTR7E5M4WFrvrclp7J8E
To: /content/rotate_model.h5
100%|██████████| 12.5M/12.5M [00:00<00:00, 17.7MB/s]


In [ ]:
# Оценка модели на проверочной выборке (500 изображений каждого угла поворота (0, 90, -90, 180))
evaluation = model.evaluate(x, y)

accuracy = evaluation[1]
print(f"Точность модели на проверочной выборке: {accuracy * 100:.2f}%")

60/60 [==============================] - 107s 2s/step - loss: 8.1022e-04 - accuracy: 1.0000
Точность модели на проверочной выборке: 100.00%


In [ ]:
# Оценка модели на проверочной выборке (500 изображений каждого угла поворота (0, 90, -90, 180))
evaluation = model.evaluate(x, y)

accuracy = evaluation[1]
print(f"Точность модели на проверочной выборке: {accuracy * 100:.2f}%")

60/60 [==============================] - 116s 2s/step - loss: 8.1022e-04 - accuracy: 1.0000
Точность модели на проверочной выборке: 100.00%


# Мусор

In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

def create_neural_network_model(input_shape, num_classes):
    # Задаем значения гиперпараметров
    image_block_1_block_type = 'vanilla'
    image_block_1_normalize = True
    image_block_1_augment = False
    image_block_1_conv_block_1_kernel_size = 3
    image_block_1_conv_block_1_num_blocks = 1
    image_block_1_conv_block_1_num_layers = 2
    image_block_1_conv_block_1_max_pooling = True
    image_block_1_conv_block_1_separable = False
    image_block_1_conv_block_1_dropout = 0.25
    image_block_1_conv_block_1_filters_0_0 = 32
    image_block_1_conv_block_1_filters_0_1 = 64
    classification_head_1_spatial_reduction_1_reduction_type = 'flatten'
    classification_head_1_dropout = 0.5
    optimizer = 'adam'
    learning_rate = 0.001

    # Создаем модель нейросети
    input_layer = Input(shape=(input_shape))

    # Создание блока свертки
    x = input_layer
    for _ in range(image_block_1_conv_block_1_num_blocks):
        for _ in range(image_block_1_conv_block_1_num_layers):
            x = Conv2D(filters=image_block_1_conv_block_1_filters_0_0 if _ == 0 else image_block_1_conv_block_1_filters_0_1,
                       kernel_size=(image_block_1_conv_block_1_kernel_size, image_block_1_conv_block_1_kernel_size),
                       padding='same', activation='relu')(x)
        if image_block_1_conv_block_1_max_pooling:
            x = MaxPooling2D(pool_size=(2, 2))(x)
        if image_block_1_conv_block_1_dropout > 0.0:
            x = Dropout(image_block_1_conv_block_1_dropout)(x)

    # Flatten или другой тип пространственного уменьшения
    if classification_head_1_spatial_reduction_1_reduction_type == 'flatten':
        x = Flatten()(x)
    # Добавление слоев для классификации
    x = Dropout(classification_head_1_dropout)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)

    # Компиляция модели с метриками Recall, Precision и F1
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', recall_m, precision_m, f1_m])

    return model

# Пример использования функции для создания модели
input_shape = (224, 224, 1)  # Форма входных изображений
num_classes = 4  # Количество классов
model = create_neural_network_model(input_shape, num_classes)
model.summary()

In [ ]:
def print_model_acc(model):
    # Оценка модели на тестовой выборке
    evaluation = model.evaluate(xTest, yTest)
    accuracy = evaluation[1]
    print(f"Точность модели на тестовой выборке: {accuracy * 100:.2f}%")

# Для сбора мусора
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

def plot_history(history):
    # График для accuracy и val_accuracy
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # График для loss и val_loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

save_path = "/content/drive/MyDrive/best_model_auto.h5"

# Создание объекта ModelCheckpoint для сохранения весов
checkpoint = ModelCheckpoint(save_path, monitor='val_accuracy', save_best_only=True, mode='max')
garbage_collector = GarbageCollectorCallback()

In [ ]:
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=5, batch_size=64, callbacks=[checkpoint])

# Загрузка лучших весов после обучения
model.load_weights(save_path)

plot_history(history)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
# Обучение модели с использованием ModelCheckpoint
history = model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=5, batch_size=32, callbacks=[checkpoint])
model.load_weights(save_path)

plot_history(history)

In [ ]:
# Оценка модели на тестовой выборке
evaluation = model.evaluate(xTest, yTest)

accuracy = evaluation[1]
print(f"Точность модели на тестовой выборке: {accuracy * 100:.2f}%")

13/13 [==============================] - 19s 1s/step - loss: 0.2388 - accuracy: 0.9949
Точность модели на тестовой выборке: 99.49%


##### **Валидационная выборка**

In [ ]:
from keras.models import load_model

# Загрузка модели
model = load_model('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/my_model.h5')

In [ ]:
# Оценка модели на проверочной выборке
evaluation = model.evaluate(x, y)

accuracy = evaluation[1]
print(f"Точность модели на проверочной выборке: {accuracy * 100:.2f}%")

61/61 [==============================] - 75s 1s/step - loss: 0.0362 - accuracy: 0.9964
Точность модели на проверочной выборке: 99.64%


In [ ]:
# Используйте метод predict для модели, чтобы получить предсказания
predictions = model.predict(prepare_img(files_dirs[0]))  # Предполагается, что ваша модель ожидает 2D массив

# Результат будет в переменной predictions, которая будет содержать предсказанные значения
print("Предсказанные значения:", predictions)

## *Проверка скорости работы лучшей модели*
> Проверяем, сколько каждого типа изображений в базе по менению сети

> **Средняя скорость обработки одного изображения - 57.75 мс.**

In [ ]:
files_dirs = [directory + '/' + i for i in os.listdir(directory)]

In [ ]:
def open_img(img_path):
    return Image.open(img_path)


def prepare_img(img_path):
    image = Image.open(img_path)
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])

    return image


def load_img(img_path, single=False):
    # Загрузка изображения и изменение размера, для модели поворота изображений
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0

    if single == True:
        img_array = np.expand_dims(img_array, axis=0)

    return img_array


def load_img_cv2(img_path, single=False):
    # Загрузка изображения и изменение размера, для модели поворота изображений
    image = cv2.imread(img_path)
    if len(image.shape) != 3 or image.shape[2] != 3:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    image = cv2.resize(image, (224, 224))
    image_array = np.array(image) / 255.0

    if single==True:
        image_array = np.expand_dims(image_array, axis=0)

    return image_array


# def prepare_imgs(img_paths):
#     return np.array([np.array(Image.open(img_path).resize((224, 224))) / 255.0 for img_path in img_paths])


def prepare_imgs(img_paths):
    return np.array([load_img_cv2(img_path) for img_path in img_paths])


def rotate_img_by_index(img, index):
    if index == 0:
        return img
    return img.rotate([0,90,-90,180][index])

In [ ]:
def rotate_img_by_index(img, index):
    if index == 0:
        return img
    return img.rotate([0,90,-90,180][index])

In [ ]:
prepare_img(files_dirs[0]).shape

(1, 224, 224, 3)

In [ ]:
from keras.models import load_model

# Загрузка модели
model = load_model('/content/drive/MyDrive/Colab/Базы/Стажировка 2023/Распознавание информации на платежных поручениях/BaseMobileNetV3/my_model.h5')

In [ ]:
wrong_list = sorted([i-1 for i in [184, 185, 171, 157, 129, 12]] + [i+200-1 for i in [162, 136, 70, 48]] + [i+400-1 for i in [78,43]] + [i+500-1 for i in [39, 27, 25]] + [i+700-1 for i in [193,79,55]] + [i+900-1 for i in [83, 60]] + [1000-1 + i for i in [246,241,240,238,208,175,114,92,85,78,51,48,46,42,40,35,1]] + [1250-1 + i for i in [245,240,188,178,135,102]])

In [ ]:
max_files = 50
print(f'Всего файлов: {len(files_dirs[:max_files])}')

# Предсказываем значения
with timex():
    predictions = model.predict(prepare_imgs(files_dirs[:max_files]))

# print("Предсказанные значения:", predictions)

Всего файлов: 50
2/2 [==============================] - 2s 624ms/step
Время обработки: 0 мин 9.34 c


In [ ]:
print(f'Всего файлов: {len(files_dirs[100:200])}')

# Предсказываем значения
x = prepare_imgs(files_dirs[100:200])
with timex():
    predictions = model.predict(x)

Всего файлов: 100
4/4 [==============================] - 4s 808ms/step
Время обработки: 0 мин 5.69 c


In [ ]:
print(f'Всего файлов: {len(files_dirs[200:500])}')

# Предсказываем значения
x = prepare_imgs(files_dirs[200:500])
with timex():
    predictions = model.predict(x)

Всего файлов: 300
10/10 [==============================] - 13s 1s/step
Время обработки: 0 мин 21.85 c


In [ ]:
print(f'Всего файлов: {len(files_dirs[500:1000])}')

# Предсказываем значения
x = prepare_imgs(files_dirs[500:1000])
with timex():
    predictions = model.predict(x)

Всего файлов: 500
16/16 [==============================] - 20s 1s/step
Время обработки: 0 мин 21.76 c


In [ ]:
((5.69 / 100) + (21.85 / 300) + (21.76 / 500)) / 3

0.057751111111111114

In [ ]:
0.05775 * 1000

57.75

##### Предсказание классов

In [ ]:
print(f'Всего файлов: {len(files_dirs)}')

# Предсказываем значения
with timex():
    predictions = model.predict(prepare_imgs(files_dirs))

In [ ]:
indices = np.argmax(predictions, axis=1)
print(indices)

In [ ]:
from collections import Counter

my_list = [1, 2, 2, 3, 3, 3, 4, 4, 4, 4]
element_count = Counter(my_list)

for element, count in element_count.items():
    print(f"{element}: {count}")